Storage Folder for videos


In [1]:
output_folder = "/mnt/Personal/Projects/Drivator"

Imported Libraries


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import random
from tqdm import tqdm
from IPython.display import clear_output
from collections import deque
import copy

Experimentable Functions


In [3]:
def normalize_radian(rad):
    # return rad
    return (rad + np.pi) % (2 * np.pi) - np.pi

def get_speed_from_slip_angle(max_speed,delta_angle,max_error_angle):
    
    max_error_angle=max(max_error_angle,delta_angle)
    
    delta_angle = abs(normalize_radian(delta_angle))
    
    speed = max_speed * (1 - np.sqrt(delta_angle/max_error_angle))
    
    # print(np.rad2deg(delta_angle),np.rad2deg(max_error_angle),speed)
    
    return speed

def get_slip_angle_from_speed(max_speed,speed,d_theta,max_error_angle):

    max_speed=max(speed,max_speed)
    
    dev_angle = abs(normalize_radian(max_error_angle*((1-(speed/max_speed))**2)))
    
    angle=d_theta
    
    if abs(d_theta)>dev_angle:
        angle = np.sign(d_theta)*dev_angle
        
    angle = normalize_radian(angle)
    
    return angle

def get_center_speed(speed):
    return speed

def get_gyro_angle(angle):
    return angle

Map Matrix Initializations


In [4]:
import numpy as np

# we have a list of constraints
# the matrix has areas numbered as index, if the particle is in that area, then its destination is that point corresponding to the index

def get_rotated_index(size, theta, i_out, j_out):

    cx = (size[0] - 1) / 2
    cy = (size[1] - 1) / 2

    # Center of rotation
    cx = (size[0] - 1) / 2
    cy = (size[1] - 1) / 2

    # Position relative to center
    x_out = i_out - cx
    y_out = j_out - cy

    # Apply inverse rotation
    cos_t = np.cos(-theta)
    sin_t = np.sin(-theta)

    x_src = cos_t * x_out - sin_t * y_out
    y_src = sin_t * x_out + cos_t * y_out

    # Back to matrix indices
    i_src = x_src + cx
    j_src = y_src + cy

    i_src = int(round(i_src))
    j_src = int(round(j_src))
    
    return i_src,j_src


class MapMatrix:
    def __init__(self, n):
        self.size = n
        self.shape = (self.size, self.size)
        
        self.map_matrix = np.full(self.shape, 0, dtype=int)
        
        self.constraints=[]        
        self.origin = (0, 0)
        self.rotation=0
        
    def index_within_bounds(self,x,y):
        x,y = get_rotated_index(self.shape,self.rotation,x,y)
        
        return (0 <= x < self.shape[0]) and (0 <= y < self.shape[1])
        
    def refill(self,n,m):
        self.shape=(n,m)
        self.map_matrix = np.full(self.shape, 0, dtype=int)
        self.shape = (n, m)
        
    def set_constraints_list(self,point_list):
        self.constraints = point_list
        
    def set_constraints_point(self,index,point):
        self.constraints[index] = point
        
    def get_constraints(self,index):
        if index==len(self.constraints):
            index-=1
        return self.constraints[index]
    
    def get_constraints_list(self):
        return self.constraints
    
    def get_last_constraint(self):
        return self.constraints[-1]
        
    def set_value(self, x, y, val):
        
        x,y = get_rotated_index(self.shape,self.rotation,x,y)
        
        if self.index_within_bounds(x,y):
            self.map_matrix[x][y] = val
        else:
            raise IndexError("Coordinates out of bounds")
            # return 0

    def get_value(self, x, y):
        
        x,y = get_rotated_index(self.shape,self.rotation,x,y)
        
        if self.index_within_bounds(x,y):
            return self.map_matrix[x][y]
        else:
            # raise IndexError("Coordinates out of bounds")
            return 0
                    
    def get_matrix(self):
        return self.map_matrix
    
    def get_orientation(self):
        return self.rotation
    
    def set_orientation(self,angle):
        self.rotation=angle

    def set_origin(self, x, y):
        self.origin = (x, y)

    def get_origin(self):
        return self.origin

    def get_size(self):
        return self.shape


map_matrix = MapMatrix(n=180)

Helper Functions

In [5]:

def index_to_point(i, j, min_dist):

    origin = map_matrix.get_origin()

    i, j = i*min_dist+origin[0], j*min_dist+origin[1]

    return i, j


def point_to_index(x, y, min_dist):

    origin = map_matrix.get_origin()
    # print(origin)

    x, y = (x-origin[0])/min_dist, (y-origin[1])/min_dist

    x, y = int(round(x)), int(round(y))

    return x, y


def round_off_step(x, min_dist):

    step = 1/min_dist

    sign = np.sign(x)
    x = abs(x)

    int_x = int(x)
    frac_x = x-int(x)

    idx = round(frac_x*step)/step

    y = sign*(int_x + idx)

    return y


Find nearest point using approximation

In [6]:
def find_intersection(pos_x, pos_y, theta, max_look, min_dist):
    

    dest_x = pos_x + np.cos(theta) * max_look
    dest_y = pos_y + np.sin(theta) * max_look

    pos_i, pos_j = point_to_index(pos_x, pos_y, min_dist)
    dest_i, dest_j = point_to_index(dest_x, dest_y, min_dist)
    
    default_value = 0 if map_matrix.get_value(pos_i,pos_j)!=0 else -1

    x0, y0, x1, y1 = pos_i, pos_j, dest_i, dest_j
    
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy

    while True:
        
        if x0 == x1 and y0 == y1:
            break
        
        nx = x0
        ny = y0
        e2 = 2 * err
        if e2 > -dy:
            err -= dy
            nx = x0 + sx
        if e2 < dx:
            err += dx
            ny = y0+sy
            
        if not map_matrix.index_within_bounds(nx,ny):
            break
        
        if map_matrix.get_value(nx,ny) == default_value:
            break
        
        x0=nx
        y0=ny
            
    ax,ay = pos_x,pos_y
    bx,by = index_to_point(x0,y0,min_dist)
    
    dist = np.sqrt((ax-bx)**2+(ay-by)**2)
    point = [bx,by]
    

    return dist,point
    

Getting points for the bounded region


In [7]:
def get_path_list(min_dist):

    size = map_matrix.get_size()
    
    x_list=[]
    y_list=[]
    
    for i in tqdm(range(size[0])):
        for j in range(size[1]):
            
            if map_matrix.get_value(i,j)!=0:
                
                x,y = index_to_point(i,j,min_dist)
                x_list.append(x)
                y_list.append(y)

    return x_list, y_list

Error Calculating Fucntions


In [8]:


def accident_theta(t_accident, curr_time, max_deviation, min_deviation):
    
    if t_accident==0:
        return 0
    
    if abs((curr_time) % (t_accident)) <= 0.01 and abs(curr_time) >= 0.01:

        if random.random() < 0.5:
            # Range [-b, -a]
            deviation_deg = random.uniform(-max_deviation, -min_deviation)
        else:
            # Range [a, b]
            deviation_deg = random.uniform(min_deviation, max_deviation)

        deviation_rad = np.deg2rad(deviation_deg)

        return deviation_rad

    return  0


def get_dist_angle(pos_x, pos_y, min_dist,dest_index):
    
    pos_i,pos_j= point_to_index(pos_x,pos_y,min_dist)
    
    temp_index=map_matrix.get_value(pos_i,pos_j)
    dest_index = temp_index if temp_index!=0 else dest_index
    
    destination = map_matrix.get_constraints(dest_index)
    dest_x,dest_y = destination[0],destination[1]

    dist = np.sqrt((dest_x-pos_x)**2 + (dest_y-pos_y)**2)
    
    angle = np.arctan2((dest_y-pos_y), (dest_x-pos_x))


    return dest_index, dist, angle


def get_boundary_values(pos_x, pos_y, theta, max_dist, min_dist):
    
    bound_angles = [theta+(np.pi/2),theta-(np.pi/2)]
    
    bounds=[]
    points=[]
    
    for a in bound_angles:
        b, p = find_intersection(pos_x,pos_y,a,max_dist,min_dist)
        bounds.append(b)
        points.append(p)

    # ratio = 1-((max_dist-bound)/max_dist)

    return bounds, points, bound_angles

def error_calculation(pos_x, pos_y, theta,dest_index, max_dist, min_dist, bound_weight,max_error_angle):
    
    dest_index,dist,dest_angle = get_dist_angle(pos_x,pos_y,min_dist,dest_index)
    
    bounds, points, bound_angles = get_boundary_values(pos_x, pos_y, theta, max_dist, min_dist,)
    
    vector_x=0
    vector_y=0
    
    att_x=np.cos(dest_angle)
    att_y=np.sin(dest_angle)
    
    rep_x=0
    rep_y=0
    
    for i in range(len(points)):
        # bound_value = max((max_dist)**2-(bounds[i])**2,0)
        # bound_dist = np.sqrt(bound_value)
        
        bound_dist = max((max_dist-bounds[i]),0)
        
        rep_x+=bound_dist*np.cos(bound_angles[i])
        rep_y+=bound_dist*np.sin(bound_angles[i])
        
    rep_value = np.sqrt(rep_x**2 + rep_y**2)+1e-5
    
    rep_x=rep_x/rep_value
    rep_y=rep_y/rep_value
    
    vector_x = att_x - bound_weight*rep_x
    vector_y = att_y - bound_weight*rep_y
        
    # vector_value = np.sqrt(vector_x**2 + vector_y**2) # value = 1
    vector_angle = np.arctan2(vector_y,vector_x)
    
    error_angle = vector_angle-theta
    error_angle = normalize_radian(error_angle)
        
    if abs(error_angle) > max_error_angle:
        error_angle = np.sign(error_angle) * max_error_angle

    return error_angle,np.min(bounds),dest_index
    
def reached_end(x,y,fin,threshold):
    
    fx,fy = fin[0],fin[1]
    
    val = np.sqrt((fx-x)**2+ (fy-y)**2)
    
    return val<=threshold

Plotting Function


In [9]:
def plotter(positions, pos_x, pos_y, y_axis, min_dist, max_trail, x_bound, y_bound, speed, bound, 
            plotter_x, plotter_y):
    
    const_points = map_matrix.get_constraints_list()
    
    fig, ax = plt.subplots(figsize=(20, 10))
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    # Customize axes
    if not y_axis:
        ax.yaxis.set_visible(False)
    else:
        ax.tick_params(axis='y', colors='white')

    ax.tick_params(axis='x', colors='white')

    # plot the ball and its trail
    ball, = ax.plot([], [], 'ro', markersize=10)
    trail, = ax.plot([], [], 'r-', linewidth=1)

    ax.set_xlim(pos_x - x_bound, pos_x + x_bound)
    ax.set_ylim(pos_y - y_bound, pos_y + y_bound)

    # update ball and trail positions
    i = len(positions) - 1
    pos_x, pos_y = positions[i]
    xs, ys = zip(*positions[max(0, i - max_trail):i + 1])

    ball.set_data([pos_x], [pos_y])
    trail.set_data(xs, ys)

    # plot the points as small white circles (marker size 0.2)
    ax.plot(plotter_x, plotter_y, 'o', color='white', markersize=0.2, markeredgecolor='white')

    # plot const_points as blue circles (marker size 2)
    if len(const_points)!=0:
        const_x, const_y = zip(*const_points)
        ax.plot(const_x, const_y, 'o', color='blue', markersize=2, markeredgecolor='blue')

    ax.text(0.95, 0.95, f"Speed = {speed:.3f}, Bound = {bound:.3f}",
            transform=ax.transAxes, fontsize=12, color='white', ha='right', va='top')

    return fig, ax


PID Class


In [10]:
# PID Controller class
class PID:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.integral = 0
        self.prev_error = 0

    def update(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.prev_error = error
            
        return output

Simulator Function


In [11]:

def simulate_particle(max_speed=10, speed_up_ratio=1.0, t_accident=2.0, delta_time=0.0, start_point=(0, 0), start_angle=0, min_dist=0.1,
                      fps=20, y_axis=True, show_gif=False, x_bound=5, y_bound=5,stab_dist=0.5, max_trail=500, boundary=25):
    
    
    plotter_x,plotter_y = get_path_list(min_dist)

    fps = fps//speed_up_ratio

    deviation_rad = 0
    
    if delta_time==0.0:
        dt = min_dist/max_speed
    else:
        dt= delta_time

    frame_wait = 1/(fps)
    frame = 0

    pos_x, pos_y = start_point[0], start_point[1]
    
    destination_point = map_matrix.get_last_constraint() 

    angle = np.deg2rad(start_angle)
    
    pid_angle_k = [30,5,0]
    pid_speed_k = [10,0,0]
    
    pid_angle = PID(Kp=pid_angle_k[0], Ki=pid_angle_k[1], Kd=pid_angle_k[2])
    
    pid_speed = PID(Kp=pid_speed_k[0], Ki=pid_speed_k[1], Kd=pid_speed_k[2])

    positions = [(pos_x, pos_y)]

    # counter=0
    
    dest_index=0
    
    speed = 0
    bound = boundary

    start_time = time.time()

    min_deviation = 4
    max_deviation = 10
    
    max_error_angle=np.deg2rad(150)
    
    max_slip_angle = np.deg2rad(160)*(pid_angle_k[0]*dt + pid_angle_k[1]*dt*dt)

    total_time = 0
    average_wait=0
    

    while True:
        
        if reached_end(pos_x,pos_y,destination_point,stab_dist):
            break
        
        
        #############
        # Accident
        
        deviation_rad = accident_theta(
            t_accident, total_time, max_deviation, min_deviation)

        angle += deviation_rad
        
        ###############
            
            
        ###############
        # Error Calculation

        pos_x, pos_y = positions[-1][0], positions[-1][1]
        
        error_angle,bound,dest_index = error_calculation(pos_x, pos_y, angle,dest_index, boundary, min_dist, bound_weight=0.35,max_error_angle=max_error_angle)
        

        ###############


        ###############
        # Movement Calculation
        
        corr_angle = pid_angle.update(error_angle, dt) * dt
        corr_angle= normalize_radian(corr_angle)
        
        
        # print(np.rad2deg(corr_angle))
        
        probable_speed = get_speed_from_slip_angle(max_speed,corr_angle,max_slip_angle)
        error_speed = probable_speed-speed
        corr_speed = pid_speed.update(error_speed,dt) * dt
        
        speed += corr_speed
        speed = min(speed,max_speed)
        
        corr_angle = get_slip_angle_from_speed(max_speed,speed,corr_angle,max_slip_angle)
        angle += corr_angle
        
        
        ##################
        
        ##################
        # Send Data to Motors
        
        ##################
        
        
        
        ##################
        # Get sensor data
        
        # print(speed)
        
        speed=get_center_speed(speed)
        angle = get_gyro_angle(angle)
        
        ###################
        
        
        
        ###################
        # Simulating path

        travel_dist = dt*speed
        
        angle = normalize_radian(angle)
        
        dx = travel_dist * np.cos(angle)
        dy = travel_dist * np.sin(angle)
        pos_x += dx
        pos_y += dy
        positions.append((pos_x, pos_y))

        total_time += dt
        
        ##################
        
        
        if total_time-(frame+1)*frame_wait > 0  and show_gif:
            
            # while(True and pos_x>2):
            #     pass
            
            clear_output(wait=True)

            frame += 1
            # print(speed)
            fig, _ = plotter(positions, pos_x, pos_y, y_axis, min_dist, max_trail, x_bound, y_bound, speed, bound,plotter_x,plotter_y)

            time_passed = time.time() - start_time
            wait = total_time/speed_up_ratio - time_passed

            average_wait = (average_wait*(frame-1)+(wait*speed_up_ratio))/frame
            
            if (wait > 0):
                time.sleep(wait)

            # fig.show()
            plt.show()
            
            

    print("Final Position -- ", pos_x, pos_y, "cm")
    print("Total Time -- ", total_time, "s")
    print("Average Wait -- ", average_wait, "s")

Making the map


In [12]:



def create_map(generate=False):
    
    func_path =output_folder+"/map_images/func_data/"
    pic_path = output_folder+"/map_images/pic_data/"
    
    
    data_path = pic_path
    
    matrix = np.load(data_path+'map_matrix.npy')
    constraints = np.load(data_path+'constraints.npy')
    origin = np.load(data_path+'origin.npy')
    
    origin = origin[0]+map_matrix.get_origin()[0],origin[1]+map_matrix.get_origin()[1]
    
    map_origin=map_matrix.get_origin()
    
    for i in range(len(constraints)):
        constraints[i] = (constraints[i][0]+map_origin[0],constraints[i][1]+map_origin[1])
    
    
    size=matrix.shape
    
    map_matrix.refill(size[0],size[1])
    
    
    if generate:
        map_matrix.set_constraints_list([constraints[0],constraints[-1]])
    else:
        map_matrix.set_constraints_list(constraints)
    # print(constraints)
    
    for i in tqdm(range(size[0])):
        for j in range(size[1]):
            
            if generate:
                if matrix[i][j]==0:
                    map_matrix.set_value(i,j,0)
                else:
                    map_matrix.set_value(i,j,1)
                    
            else:
                map_matrix.set_value(i,j,matrix[i][j])
            
    return origin

def rotate_point(point, center, theta):
    
    x,y = point[0],point[1]
    cx,cy = center[0],center[1]
    
    # Translate point to origin (relative to center)
    x_rel = x - cx
    y_rel = y - cy
    
    # Apply rotation matrix
    x_rot = x_rel * np.cos(theta) - y_rel * np.sin(theta)
    y_rot = x_rel * np.sin(theta) + y_rel * np.cos(theta)
    
    # Translate back
    x_new = x_rot + cx
    y_new = y_rot + cy
    
    return x_new, y_new

 
def rotate_cosntraints(constraints,angle,min_dist):
    
    origin=map_matrix.get_origin()
    size = map_matrix.get_size()
    
    center = origin[0]+(size[0]*min_dist)/2,origin[1]+(size[1]*min_dist)/2
    
    # print(center)
    
    rot_con=[]
    
    for const in constraints:
        nx,ny = rotate_point(const,center,angle)
        
        rot_con.append((nx,ny))
        
    return rot_con

def rotate_map(angle,min_dist):
    
    map_matrix.set_orientation(angle)
    
    const = map_matrix.get_constraints_list()
    rot_con = rotate_cosntraints(const,angle,min_dist)
    map_matrix.set_constraints_list(rot_con)
    
    return rot_con[0]
    
    # print(rot_con)
    
    

Execution


In [ ]:
start_point = create_map(generate=False)
# start_point=(10,0)

accuracy_dist = 0.1

target_speed = 10

vel = 10

# start_point = rotate_map(np.deg2rad(90),min_dist=accuracy_dist)

simulate_particle(max_speed=vel,
                speed_up_ratio=target_speed/vel,
                delta_time=0.001,
                t_accident=0,
                start_point=start_point,
                start_angle=45,
                min_dist=accuracy_dist,
                fps=20,
                y_axis=True,
                show_gif=True,
                x_bound=40,
                y_bound=20,
                stab_dist=0.5,
                max_trail=200,
                boundary=4)

  5%|▌         | 108/2000 [00:01<00:22, 85.89it/s]